In [ ]:
# pip (în kernelul curent)
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "openpyxl"])


In [3]:
import importlib, super8_indicators, super8_short_backtest
importlib.reload(super8_indicators)
importlib.reload(super8_short_backtest)
import pandas as pd
from super8_short_backtest import (
    Super8ShortBacktester,
    ShortParams,
    RiskSettings,
    compute_spread,
    compute_performance_metrics,
    print_performance_summary,
)
import numpy as np

# 1) Încarcă datele tale (2h, cu price/high/low/volume; Return/Spread sunt calculate automat dacă lipsesc)
df = pd.read_csv("Simboluri_Binance/AVAXUSDT.csv")
# === Cost per tranzacție (per parte) ===
# Alege unul din profilele de mai jos:

per_side_cost_log = compute_spread(
    df,
    pay_fees_with_bnb=True,
    bar_length="H2",
    slip_floor=0.00005,
    slip_cap=0.0040,
)
per_side_cost_pct = 1.0 - np.exp(-per_side_cost_log)

print(f"Cost per parte: {per_side_cost_pct*100:.4f}% "
      f"(round-trip ~ {2*per_side_cost_pct*100:.4f}%)")

# 2) Parametrii indicatori (aliniați cu Versiune-2)
indicator_params = dict(
    sEma_Length=418, fEma_Length=58,
    ADX_len=13, ADX_smo=13, th=6.26,
    Sst=0.10, Sinc=0.04, Smax=0.40,
    MACD_options="MACD",   # pune "MAC-Z" ca să folosești varianta MAC-Z
    fastLength=20, slowLength=42, signalLength=12,
    lengthz=14, lengthStdev=16, A=-0.1, B=0.5,        # folosite doar dacă MACD_options="MAC-Z"
    volume_f=0.87, sma_Length=81,
    BB_Length=100, BB_mult=10,
    bbMinWidth01=9.3, bbMinWidth02=0.0,
    DClength=82
)

# 3) Parametrii strategiei short (TP/SL ca în Pine, conform Versiune-2)
short_params = ShortParams(
    Position="Short",            # sau "Both" dacă vrei să permiți și long-urile
    TP_options="Both",          # "Normal" | "Donchian" | "Both"
    SL_options="Both",          # "Normal" | "ATR" | "Both"
    tp=3.6, sl=3.0,
    atrPeriodSl=290, multiplierPeriodSl=320,
    trailOffset=0.38,            # trailing ignorat în implementarea simplă
    start_time=None              # sau pd.Timestamp("2022-01-01", tz="UTC")
)

# 4) Profil de risc (aliniat cu varianta live)
risk_cfg = RiskSettings(
    enabled=False,
    base_pct=0.01,
    min_pct=0.0025,
    cap_pct=0.073,
    cap_buffer_pct=0.05,
    dd_stop_pct=0.075,
    equity_start=1.0,
)

engine = Super8ShortBacktester(indicator_params, short_params, use_spread=True, log_return=True, risk=risk_cfg)
bt = engine.run(df)

metrics = compute_performance_metrics(bt, initial_equity=1.0, log_returns=engine.log_return)
print_performance_summary("AVAXUSDT", metrics)

risk_cols = [c for c in bt.columns if c.startswith("risk_")]
risk_summary_cols = [
    "risk_equity_start",
    "risk_equity_end",
    "risk_r_new",
    "risk_entry_allowed",
    "risk_entry_reason",
    "risk_dd",
    "risk_cap_hit",
]
risk_summary_cols = [c for c in risk_summary_cols if c in bt.columns]

cols_base = ["high","low","Price","volume","Return","Spread","position","trades","cstrategy","creturn"]
cols = cols_base + [c for c in risk_cols if c not in cols_base]

bt_x = bt.copy()
if isinstance(bt_x.index, pd.DatetimeIndex) and bt_x.index.tz is not None:
    # păstrează valorile în UTC, doar scoate informația de timezone
    bt_x.index = bt_x.index.tz_localize(None)

with pd.ExcelWriter("s8_short_backtest_view.xlsx") as writer:
    bt_x[cols].to_excel(writer, sheet_name="backtest", index=True)
    pd.DataFrame([metrics]).to_excel(writer, sheet_name="metrics", index=False)

bt.to_csv("s8_short_backtest_ETHUSDT.csv", index=True)

print(
    "Perf:", bt["cstrategy"].iloc[-1],
    " | Buy&Hold:", bt["creturn"].iloc[-1],
    " | Trades:", int(bt["trades"].sum()),
    " | Profit factor:", metrics.get("profit_factor"),
    " | Max DD%:", metrics.get("max_drawdown_pct"),
)

if risk_summary_cols:
    print("Ultimele 10 valori pentru diagnosticele de risc:")
    print(bt[risk_summary_cols].tail(10))
else:
    print("Nu am găsit coloane de risc în rezultat (verifică RiskSettings).")

Cost per parte: 0.0550% (round-trip ~ 0.1100%)
[BT] df rows=19731 | cols=['high', 'low', 'Price', 'volume', 'Return', 'Spread']
[BT] time: 2021-06-01 02:00:00+00:00 → 2025-12-01 12:00:00+00:00 | dup_time=0
[IND] rows=19731 | cols=['sEMA', 'fEMA', 'EMA_longCond', 'EMA_shortCond', 'adx', 'di_plus', 'di_minus', 'ADX_longCond']... | has_BB=True | has_hist=True | has_histmacz=True
[BT] type(ind)=<class 'pandas.core.frame.DataFrame'> | isNone=False
[BT] ind rows=19731
[BT] base rows=19731 | same_index=True
len(df)= 19731  | len(ind)= 19731  | len(base)= 19731


C:\Users\Stefan\Desktop\Curs Trading\Strategii\Super8\BackTest\super8_short_backtest.py:261: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  entry_sig = entry_cond.shift(1).fillna(False).astype(bool)  # intrăm next-bar


[BT] rates → EMA_short=0.529, ADX_short=0.550, SAR_short=0.501, MACD_short=0.490, VOL_short=0.435, BB_short01=0.000, entry=0.072
[BT] first TRUE EMA_short: 2021-06-01 16:00:00+00:00
[BT] first TRUE ADX_short: 2021-06-01 08:00:00+00:00
[BT] first TRUE SAR_short: 2021-06-01 04:00:00+00:00
[BT] first TRUE MACD_short: 2021-06-01 08:00:00+00:00
[BT] first TRUE VOL_short: 2021-06-01 02:00:00+00:00
[BT] first TRUE entry: 2021-06-07 14:00:00+00:00
[BT] summary → rows=19731, trades=765, pos_rate=0.101, perf=423.9745, bh=0.7191
Rezumat performanță pentru AVAXUSDT:
  Total P&L: 422.97 (42297.45%)
  Max drawdown: 12.68%
  Profit factor: 4.67
  Win rate: 56.28% din 382 tranzacții
  Durată maximă tranzacție: 1.83 zile
Perf: 423.97451237906296  | Buy&Hold: 0.7190636303700343  | Trades: 765  | Profit factor: 4.667632813543823  | Max DD%: 12.680538248562359
Ultimele 10 valori pentru diagnosticele de risc:
                           risk_equity_start  risk_equity_end  risk_r_new  \
time                 

In [ ]:
import pandas as pd
df_view = pd.read_excel("s8_short_backtest_view.xlsx")
df_view.tail()
